In [39]:
# Import modules
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [40]:
#load data and take out id columns
new_data = pd.read_csv('data/model_data.csv')
new_data1 = pd.read_csv('data/model_data.csv')
droplis = ['Unnamed: 0','encounter_id','patient_nbr']
new_data = new_data.drop(droplis, axis=1)

In [41]:
#find numeric columns
new_data._get_numeric_data().columns

Index(['admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'target', 'self_pay',
       'medicaid/medicare', 'coverByInsurance', 'clustering3', 'clustering4',
       'clustering2'],
      dtype='object')

In [42]:
#numeric columns
num =['time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency',
'number_inpatient','number_diagnoses']


In [43]:
#normalize new_data numeric columns
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(new_data[num])
new_data[num] = transformer.transform(new_data[num])
new_data[num] = pd.DataFrame(new_data[num], columns= new_data1[num].columns)

In [44]:
#enlabel the columns
new_data = new_data
new_data['admission_type_id'] = pd.Categorical(new_data.admission_type_id)
new_data['discharge_disposition_id'] = pd.Categorical(new_data.discharge_disposition_id)
new_data['admission_source_id'] = pd.Categorical(new_data.admission_source_id)

cat_cols = list(new_data.select_dtypes(include=[object]).columns)
for col in cat_cols:
   new_data[col] = pd.Categorical(new_data[col])

le = preprocessing.LabelEncoder()

col_to_encode = new_data[list(new_data.select_dtypes(include=['category']).columns)]
for col in col_to_encode:
   new_data[col] = le.fit_transform(new_data[col])

In [45]:
new_data['admission_type_id'] = pd.Categorical(new_data.admission_type_id)
new_data['discharge_disposition_id'] = pd.Categorical(new_data.discharge_disposition_id)
new_data['admission_source_id'] = pd.Categorical(new_data.admission_source_id)
cat_cols = list(new_data.select_dtypes(include=[object]).columns)
for col in cat_cols:
  new_data[col] = pd.Categorical(new_data[col])

enc = OneHotEncoder(handle_unknown='ignore')

col_to_encode = new_data[list(new_data.select_dtypes(include=['category']).columns)]
for col in col_to_encode:
  new_data[col] = enc.fit_transform(new_data[col])

In [46]:
#Spliting to training and testing datasets
target = new_data.target
X_train, X_test, y_train, y_test = train_test_split(new_data, target, test_size=0.25, stratify = target)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(64875, 52) (64875,)
(21625, 52) (21625,)


In [47]:
##### Up-sample logistic regression  #####

In [48]:
#Up-sample X_train
from sklearn.utils import resample
train_x_majority = X_train[X_train.target==0]
train_x_minority = X_train[X_train.target==1]
train_x_minority_upsampled = resample(train_x_minority,replace=True,n_samples=57558,random_state=123)
x = pd.concat([train_x_majority, train_x_minority_upsampled])


In [67]:
#Up-sample y_train
y_train = pd.DataFrame(y_train)
train_y_majority = y_train[y_train.target==0]
train_y_minority = y_train[y_train.target==1]
train_y_minority_upsampled = resample(train_y_minority,replace=True,n_samples=57558,random_state=123)
y = pd.concat([train_y_majority, train_y_minority_upsampled])
y.target.value_counts()

0    57635
1    57558
Name: target, dtype: int64

In [54]:
#drop target from training
x = x.drop('target', axis=1)

In [66]:
## Fit the model with training
decision_logit = LogisticRegression(C=0.01, penalty= 'l1', random_state=0)
decision_logit.fit(x, y)

## The score (accuracy for classification problems):
decision_logit.score(x, y)

/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5880392037710624

In [56]:
#up-sample AUC for training 
from sklearn.metrics import roc_auc_score
upsample_AUC_train =roc_auc_score(y, decision_logit.predict_proba(x)[:,1])
upsample_AUC_train

0.6256510999079499

In [37]:
#drop target from test
#X_test = X_test.drop('target', axis=1)

In [16]:
## Fit the model with testing
decision_logit1 = LogisticRegressionCV(cv=5, random_state=0, multi_class='multinomial')
decision_logit1.fit(X_test, y_test)

## The score (accuracy for classification problems):
decision_logit1.score(X_test, y_test)

0.8890635838150289

In [17]:
#AUC for testing
from sklearn.metrics import roc_auc_score
upsample_AUC_test=roc_auc_score(y_test, decision_logit.predict_proba(X_test)[:,1])
upsample_AUC_test

0.6190443008106945

In [64]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
grid={"C":np.logspace(-3,3,7), "penalty":["l1",'l2']}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=5)
logreg_cv.fit(X_train,y_train)

/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]), 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [62]:
logreg_cv.best_params_

{'C': 0.01, 'penalty': 'l1'}

In [53]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, decision_logit(X_test))


TypeError: 'LogisticRegressionCV' object is not callable